# TP 1
**Aluno**: Ariel Augusto dos Santos

**Matrícula**: 2017068726

_Questões_: 1, 7, 9, 8, 3, 10

In [1]:
import csv
import io
import sqlite3
import pandas as pd

db_conn = sqlite3.connect('./despesas.db')
db_exists = db_conn.cursor().execute("SELECT COUNT(*) FROM sqlite_master WHERE type='table';").fetchone()[0]
if not db_exists:
    db_sql = io.open('./despesas.sql', 'r').read()
    db_conn.executescript(db_sql)

csv_file = io.open('2017068726.csv', 'w', newline='')
csv_writer = csv.writer(csv_file, delimiter='|', quoting=csv.QUOTE_NONE)

def consulta(n, sql):
    sql = " ".join(sql.split())
    query = pd.read_sql_query(sql, db_conn)
    rows, cols = query.shape
    csv_writer.writerow([n, rows, cols, sql])
    return query

### 1. Liste todos os códigos e nomes de subfunções da função que possui o maior número de subfunções.

In [2]:
consulta(1, """
SELECT SUBFUNCAO.COD_SUBFUNCAO, NOME_SUBFUNCAO
FROM SUBFUNCAO
         INNER JOIN DETALHAMENTO_FUNCAO DF
                    on SUBFUNCAO.COD_SUBFUNCAO = DF.COD_SUBFUNCAO
         INNER JOIN (SELECT COUNT(COD_SUBFUNCAO) AS CCS, COD_FUNCAO
                     FROM DETALHAMENTO_FUNCAO
                     GROUP BY COD_FUNCAO
                     ORDER BY CCS DESC
                     LIMIT 1) F
                    on F.COD_FUNCAO = DF.COD_FUNCAO;
""")

,COD_SUBFUNCAO,NOME_SUBFUNCAO
0,125,Normatização e fiscalização
1,126,Tecnologia da informação
2,131,Comunicação social
3,244,Assistência comunitária
4,306,Alimentação e nutrição
5,365,Educação infantil
6,511,Saneamento básico rural


### 7. Liste os órgãos governamentais e a quantidade de programas de governo distintos envolvidos nas despesas públicas desses órgãos.  Nesta consulta, desconsidere valores de despesa negativos ou iguais a zero no período.

In [3]:
consulta(7, """
SELECT NOME_ORGAO, COUNT(NOME_PROGRAMA_GOVERNO) AS QUANT FROM ORGAO
    JOIN DESPESA_PUBLICA DP
        on ORGAO.COD_ORGAO = DP.COD_ORGAO AND DP.VALOR_DESPESA > 0
    JOIN PROGRAMA_GOVERNO PG
        on PG.COD_PROGRAMA_GOVERNO = DP.COD_PROGRAMA_GOVERNO
    GROUP BY NOME_ORGAO;
""")

,NOME_ORGAO,QUANT
0,"Ministério da Agricultura, Pecuária e Abastec",4
1,Ministério da Defesa,2
2,Ministério da Educação,10
3,Ministério da Saúde,3
4,Ministério do Desenvolvimento Social,2


### 9. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor)  em  que  o  órgão  responsável  é  aquele  que  possui  o  maior  número  de  órgãos subordinados.

In [4]:
consulta(9, """
SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA
FROM DESPESA_PUBLICA
         INNER JOIN ORGAO O
              on O.COD_ORGAO = DESPESA_PUBLICA.COD_ORGAO AND
                 O.QTD_ORGAOS_SUBORDINADOS = (SELECT max(QTD_ORGAOS_SUBORDINADOS) FROM ORGAO);
""")

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,196,10,2018,Reserva de Contingência,0.0
1,197,1,2018,Reserva de Contingência,0.0
2,198,2,2018,Reserva de Contingência,0.0
3,199,3,2018,Reserva de Contingência,0.0
4,200,4,2018,Reserva de Contingência,0.0
...,...,...,...,...,...
289,485,8,2018,Reserva de Contingência,2400.0
290,486,9,2018,Reserva de Contingência,0.0
291,487,10,2018,Reserva de Contingência,0.0
292,488,11,2018,Reserva de Contingência,0.0


### 8. Liste os nomes das subfunções das despesas públicas envolvidas para cada programa de governo.  Sua consulta deve retornar o nome do programa de governo, a subfunção da despesa, a quantidade de entradas de despesas públicas e o valor total dessas despesas.

In [5]:
consulta(8, """
SELECT NOME_PROGRAMA_GOVERNO, NOME_SUBFUNCAO, COUNT(COD_DESPESA), SUM(VALOR_DESPESA)
FROM DESPESA_PUBLICA
         JOIN PROGRAMA_GOVERNO PG
              on PG.COD_PROGRAMA_GOVERNO = DESPESA_PUBLICA.COD_PROGRAMA_GOVERNO
         JOIN DETALHAMENTO_FUNCAO DF
              on DF.COD_DETALHAMENTO = DESPESA_PUBLICA.COD_DETALHAMENTO
         JOIN SUBFUNCAO S on S.COD_SUBFUNCAO = DF.COD_SUBFUNCAO
GROUP BY NOME_PROGRAMA_GOVERNO, NOME_SUBFUNCAO;
""")

,NOME_PROGRAMA_GOVERNO,NOME_SUBFUNCAO,COUNT(COD_DESPESA),SUM(VALOR_DESPESA)
0,BRASIL SEM MISERIA,Alimentação e nutrição,112,100373.55
1,BRASIL SEM MISERIA,Assistência comunitária,117,227883.77
2,BRASIL SEM MISERIA,Comunicação social,1,0.00
3,BRASIL SEM MISERIA,Difusão do conhecimento científico e tecnológico,13,0.00
4,BRASIL SEM MISERIA,Educação infantil,2,0.00
...,...,...,...,...
60,VIVER SEM LIMITE,Desenvolvimento tecnológico e engenharia,11,0.00
61,VIVER SEM LIMITE,Educação básica,1,0.00
62,VIVER SEM LIMITE,Ensino superior,19,0.00
63,VIVER SEM LIMITE,Normatização e fiscalização,11,0.00


### 3. Liste todos os nomes dos órgãos e o valor das despesas totais no ano de 2018 para os mesmos, ordenando de forma decrescente pelo valor.

In [6]:
consulta(3, """
SELECT NOME_ORGAO, SUM(VALOR_DESPESA) AS SOMA
FROM DESPESA_PUBLICA
         LEFT JOIN ORGAO O
              on O.COD_ORGAO = DESPESA_PUBLICA.COD_ORGAO
WHERE ANO_DESPESA = 2018
GROUP BY O.COD_ORGAO ORDER BY SOMA DESC;
""")

,NOME_ORGAO,SOMA
0,Ministério da Defesa,353376.70
1,Ministério do Desenvolvimento Social,278586.35
2,"Ministério da Agricultura, Pecuária e Abastec",90836.51
3,Ministério da Educação,64007.82
4,Ministério da Saúde,50971.19
5,Ministério dos Direitos Humanos,0.00
6,Ministério da Integração Nacional,0.00
7,Ministério do Meio Ambiente,0.00
8,"Ministério dos Transportes, Portos e Aviação",0.00
9,Ministério das Relações Exteriores,0.00


### 10. Liste todas as despesas públicas (código da despesa, mês e ano da despesa, modalidade, valor) em que o órgão responsável é aquele que possui o maior valor total em despesas durante o ano de 2018.

In [7]:
consulta(10, """
SELECT COD_DESPESA, MES_DESPESA, ANO_DESPESA, MODALIDADE_DESPESA, VALOR_DESPESA
FROM DESPESA_PUBLICA
WHERE COD_ORGAO = (
    SELECT COD_ORGAO
    FROM DESPESA_PUBLICA
    WHERE ANO_DESPESA = 2018
    GROUP BY COD_ORGAO
    ORDER BY SUM(VALOR_DESPESA) DESC
    LIMIT 1
);
""")

,COD_DESPESA,MES_DESPESA,ANO_DESPESA,MODALIDADE_DESPESA,VALOR_DESPESA
0,827,1,2018,Reserva de Contingência,0.0
1,828,2,2018,Reserva de Contingência,0.0
2,829,3,2018,Reserva de Contingência,0.0
3,830,4,2018,Reserva de Contingência,0.0
4,831,5,2018,Reserva de Contingência,0.0
5,832,6,2018,Reserva de Contingência,0.0
6,833,7,2018,Reserva de Contingência,0.0
7,834,8,2018,Reserva de Contingência,0.0
8,835,9,2018,Reserva de Contingência,334126.7
9,836,10,2018,Reserva de Contingência,0.0


### 12.
$$\pi_{NOME\_FUNCAO, NOME\_SUBFUNCAO}(FUNCAO \bowtie DETALHAMENTO\_FUNCAO \bowtie SUBFUNCAO)$$

In [8]:
consulta(12, """
SELECT NOME_FUNCAO, NOME_SUBFUNCAO
FROM FUNCAO
NATURAL JOIN DETALHAMENTO_FUNCAO
NATURAL JOIN SUBFUNCAO;
""")

,NOME_FUNCAO,NOME_SUBFUNCAO
0,Essencial à justiça,Comunicação social
1,Defesa nacional,Administração geral
2,Defesa nacional,Defesa terrestre
3,Segurança pública,Administração geral
4,Segurança pública,Policiamento
5,Segurança pública,Defesa civil
6,Segurança pública,Informação e inteligência
7,Assistência social,Normatização e fiscalização
8,Assistência social,Tecnologia da informação
9,Assistência social,Comunicação social


### 13.
$$\pi_{MODALIDADE\_DESPESA,VALOR\_DESPESA,NOME\_PROGRAMA\_GOVERNO}(PROGRAMA\_GOVERNO \bowtie \sigma_{MODALIDADE\_DESPESA="Reserva\ de\ Contingência"}(DESPESA\_PUBLICA)$$

In [9]:
consulta(13, """
SELECT MODALIDADE_DESPESA, VALOR_DESPESA, NOME_PROGRAMA_GOVERNO
FROM PROGRAMA_GOVERNO
NATURAL JOIN (
    SELECT * FROM DESPESA_PUBLICA WHERE MODALIDADE_DESPESA = 'Reserva de Contingência'
);
""")

,MODALIDADE_DESPESA,VALOR_DESPESA,NOME_PROGRAMA_GOVERNO
0,Reserva de Contingência,0.0,BRASIL SEM MISERIA
1,Reserva de Contingência,0.0,BRASIL SEM MISERIA
2,Reserva de Contingência,0.0,BRASIL SEM MISERIA
3,Reserva de Contingência,0.0,BRASIL SEM MISERIA
4,Reserva de Contingência,0.0,BRASIL SEM MISERIA
...,...,...,...
671,Reserva de Contingência,0.0,BRASIL SEM MISERIA
672,Reserva de Contingência,0.0,BRASIL SEM MISERIA
673,Reserva de Contingência,0.0,BRASIL SEM MISERIA
674,Reserva de Contingência,0.0,BRASIL SEM MISERIA


### 14.
$$\pi_{VALOR\_DESPESA, NOME\_ORGAO}(DESPESA\_PUBLICA \bowtie \sigma_{NOME\_ORGAO='Ministério\ da\ Fazenda'}(ORGAO))$$

In [10]:
consulta(14, """
SELECT VALOR_DESPESA, NOME_ORGAO
FROM (DESPESA_PUBLICA
    NATURAL JOIN (
        SELECT *
        FROM ORGAO
        WHERE NOME_ORGAO = 'Ministério da Fazenda'
    )
);
""")

,VALOR_DESPESA,NOME_ORGAO
0,0,Ministério da Fazenda
1,0,Ministério da Fazenda
2,0,Ministério da Fazenda
3,0,Ministério da Fazenda
4,0,Ministério da Fazenda
5,0,Ministério da Fazenda
6,0,Ministério da Fazenda
7,0,Ministério da Fazenda
8,0,Ministério da Fazenda
9,0,Ministério da Fazenda


### 15.
$$\pi_{NOME\_PROGRAMA\_GOVERNO, VALOR\_DESPESA}(DESPESA\_PUBLICA \bowtie \sigma_{NOME\_ORGAO='Ministério\ da\ Saúde'}(ORGAO \sigma PROGRAMA\_GOVERNO)$$

In [11]:
consulta(15, """
SELECT NOME_PROGRAMA_GOVERNO, VALOR_DESPESA
FROM DESPESA_PUBLICA
NATURAL JOIN (SELECT * FROM ORGAO WHERE NOME_ORGAO='Ministério da Saúde')
NATURAL JOIN PROGRAMA_GOVERNO;
""")

,NOME_PROGRAMA_GOVERNO,VALOR_DESPESA
0,VIVER SEM LIMITE,0.0
1,VIVER SEM LIMITE,0.0
2,VIVER SEM LIMITE,0.0
3,VIVER SEM LIMITE,0.0
4,VIVER SEM LIMITE,0.0
...,...,...
214,MAIS MEDICOS,0.0
215,MAIS MEDICOS,0.0
216,MAIS MEDICOS,0.0
217,MAIS MEDICOS,0.0


In [12]:
csv_file.close()
db_conn.close()